In [204]:
import numpy as np
import matplotlib.pyplot as plt
import math
#matplotlib inline

# Các lớp biểu diễn

In [205]:
#-------------------------------------------------------------------------------
# Lớp biểu diễn các toạ độ (x, y)
#-------------------------------------------------------------------------------
class Position:
    # Constructor
    def __init__(self, x, y):
        self.x = x
        self.y = y
#-------------------------------------------------------------------------------
# Lớp biểu diễn các ô
#-------------------------------------------------------------------------------
class Cell: ## This class represent a state
    # Constructor
    def __init__(self, position: Position, status: str, g: float = 0, parent: list = [],  h: float = 0, f: float = 0):
        self.position = position # (x, y)
        self.parent   = parent   # parent (object) in a path
        self.status   = status   # {'free', 'start', 'dirty', 'clean'}
        
        self.g        = 1 if self.status == 'dirty' else 0  # current   cost #move cost
        self.h        = h # estimated cost
        # self.costdirty = 1 if status == 'dirty' else None
        self.f        = f #fullpathcost
    #---------------------------------------------------------------------------

    def fullpathcost(self):
        return self.f
    
    def __str__(self):
        return (self.position.x, self.position.y)
    

In [206]:
class VacuumProblem():
    # Constructor
    def __init__(self, matrix: np.array, robot_inital: Cell, initial_state: list[Cell], heufunc: str = 'Euclid'):
        self.matrix = matrix
        self.robot_inital = robot_inital
        self.initial_state = initial_state ##position of dirty cells
        self.numdirty = len(self.initial_state)
        self.heufunc = heufunc
        
    def is_goal(self) -> bool: #done testing
        '''Return True if all dirty cells are cleaned and No otherwise'''
        if self.numdirty == 0:
            return True
        else:
            return False
    
    def actions(self, state: Cell) -> str|list: #done testing
        '''Return a list of possible actions'''
        move = ['Up', 'Down', 'Left', 'Right', 'LeftUp', 'RightUp', 'LeftDown', 'RightDown']
        impossible = []
        m, n = len(self.matrix), len(self.matrix[0])

        if state.position.x == 0 and state.position.y in range(0,n): ##First row 
            if state.position == (0,0):
                impossible = ['Left', 'Up', 'LeftDown', 'RightUp']
            elif state.position == (0, n-1):
                impossible = ['Up', 'LeftUp', 'RightUp', 'RightDown']
            else:
                impossible = ['Up', 'LeftUp', 'RightUp']

        elif state.position.x == (m - 1) and state.position.y in range(0, n): ##Last row
            if state.position == (m, 0):
                impossible = ['Left', 'Down', 'LeftUp', 'LeftDown', 'RightDown']
            elif state.position == (m-1, n-1):
                impossible = ['RightUp', 'LeftDown', 'Down', 'RightDown']
            else:
                impossible = ['Down', 'LeftDown', 'RightDown']


        elif state.position.y == 0 and state.position.x in range(1, m-1): #First column
            impossible = ['Left', 'LeftDown', 'LeftUp']

        elif state.position.y == (n-1) and state.position.x in range(1, m-1): ##Last column
            impossible = ['Right', 'RightUp', 'RightDown']

        if state.status == 'dirty':
            return 'Suck'
        elif len(impossible) != 0:
            for im_act in impossible:
                move.remove(im_act)
            return move
        else:
            return move
   
    def result(self, state: Cell, action: str) -> Cell: #done testing
        '''Return a new state after applying an action'''
        # newstate = state
        # Create new position object
        new_position = Position(state.position.x, state.position.y)
        # Create new state with the new position
        newstate = Cell(position=new_position, status=state.status, g=state.g)
        if action == 'Suck':
        # if state.status == 'dirty' and action == 'Suck':
            newstate.status = 'clean'
            # newstate.g += 1 ##Cost of suck
            self.numdirty -= 1 ##only calculate g(n) cost
            return newstate

        else:
            # for act in action:  ##only calculate g(n) cost
            if action == 'Up':
                newstate.position.x -= 1
            elif action == 'Down':
                newstate.position.x += 1
            elif action == 'Left':
                newstate.position.y -= 1
            elif action == 'Right':
                newstate.position.y += 1

            elif action == 'LeftUp':
                newstate.position.x -= 1
                newstate.position.y -= 1
            elif action == 'LeftDown':
                newstate.position.x += 1
                newstate.position.y -= 1
            elif action == 'RightUp':
                newstate.position.x -= 1
                newstate.position.y += 1
            elif action == 'RightDown':
                newstate.position.x += 1
                newstate.position.y += 1
            # newstate.g += 1
            newstate.parent.append(state)
            return newstate
                
    def cost(self, state, action, state2) -> int:
        '''Return cost of applying action to move from state to state 2'''
        return 1 ##move and suck have cost 1 
    
    def heuristic(self, state: Cell, othercell: Cell) -> float:
        '''Return heuristic cost between a cell and other cell'''
        if self.heufunc == 'Euclid':  ##Eucledian
            return ((state.position.x - othercell.position.x)**2 + (state.position.y - othercell.position.y)**2)**(1/2)
        elif self.heufunc == 'Manhattan':
            return abs(state.position.x - othercell.position.x) + abs(state.position.y - othercell.position.y)
    
    # def dirty2clean(self, state: Cell, otherdirty: list[Cell]):
    #     "Choose the next dirty cell to be cleaned"
    #     result = None
    #     for dirty in otherdirty:



# Algorithm

In [207]:
# def astar(problem: VacuumProblem):
#     vacuum_world = problem
#     robot = problem.robot_inital
#     dirtycells = problem.initial_state
#     dirtycells.sort(reverse = True, key = lambda cell: cell.f) ##Turn into PriorityQueue
#     open = dirtycells ##List of dirty cells to be cleaned
#     # closed = [] ##List of dirty cells that are cleaned
#     possible_acts = vacuum_world.actions(robot)

#     while len(open) != 0:
#         superdirt = open[0]
#         # open.remove(superdirt)

#         if possible_acts == 'Suck':
#             print('haha')
#             # print(open)
#             cleaned = vacuum_world.result(superdirt, 'Suck')
#             # robot.costdirty += cleaned.costdirty
#             cleaned.status = 'clean'
#             print("superdirt", superdirt.g)
#             print("robot", robot.g)
#             robot.g += superdirt.g ##Update g(n) cost
#             superdirt = cleaned ##ensure that the cell has the status 'clean' before removed
#             possible_acts = vacuum_world.actions(robot)
#             print(possible_acts)
#             open.remove(superdirt)

#         else:
#             movecost = {}
#             for action in possible_acts:
#                 newstate = vacuum_world.result(robot, action)
#                 cost = newstate.g + vacuum_world.heuristic(newstate, superdirt) + superdirt.g
#                 movecost[action] = cost
#             #Find move that has lowest cost
#             print(movecost)
#             decided = min(movecost, key = lambda k: movecost[k])
#             #Move robot
#             # previous = robot
#             robot = vacuum_world.result(robot, decided)

#             # robot.parent.append(previous)
#             robot.g += 1
#             robot.h += vacuum_world.heuristic(robot, superdirt)
#             robot.f = robot.g + robot.h 

#             #Update cost of dirty cells
#             for remain in dirtycells:
#                 print(remain.g)
#                 remain.g += 1
#             print(decided)
#         #Check if the robot has arrived at the dirty cell
#         if robot.position.x == superdirt.position.x and robot.position.y == superdirt.position.y and superdirt.status == 'dirty':
#             print("buc qua")
#             possible_acts = vacuum_world.actions(superdirt)
#     return robot

    


In [208]:
# def astar(problem: VacuumProblem): #Move to one of 8 adjacent cells
#     vacuum_world = problem
#     robot = problem.robot_inital
#     dirtycells = problem.initial_state
#     closed = [] ##List of dirty cells that are cleaned
#     possible_acts = vacuum_world.actions(robot)

#     while vacuum_world.numdirty != 0:
#         decided = None  #A tuple, first value is action, second is the dirty cell
#         if possible_acts == 'Suck':
#             cleaned = vacuum_world.result(decided[1], 'Suck')
#             robot.g += decided[1].g ##Update g(n) cost, cost of sucking the dirty cell
#             possible_acts = vacuum_world.actions(robot)
#             dirtycells.remove(decided[1])
#             closed.append(cleaned)

#         else:
            
#             movecost = {} #key is a tuple (action, goal), value is the cost
#             for action in possible_acts:
#                 for dirty in dirtycells:
#                     newstate = vacuum_world.result(robot, action)
#                     cost = newstate.g + vacuum_world.heuristic(newstate, dirty) + dirty.g
#                     movecost[(action,dirty)] = cost

#             #Find move that has lowest cost
#             decided = min(movecost, key = lambda k: movecost[k])

#             #Move robot ('Up', 'Dirtycell'): 123
#             robot = vacuum_world.result(robot, decided[0])

#             # robot.g += 1 already calculated in VacuumProblem.result()
#             robot.h += vacuum_world.heuristic(robot, decided[1])
#             robot.f = robot.g + robot.h 

#             #Update cost of dirty cells
#             for remain in dirtycells:
#                 remain.g += 1

#             #Check if the robot has arrived at the dirty cell
#             possible_acts = vacuum_world.actions(robot)
#         print(decided)
#     return robot

In [209]:
# def astar(problem: VacuumProblem):
#     """
#     A* algorithm for vacuum cleaner problem
#     Returns the path taken by the robot and total cost
#     """
#     robot = problem.robot_inital
#     path = []  # Store the path taken
#     total_cost = 0
    
#     # Continue until all dirty cells are cleaned
#     while not problem.is_goal():
#         # Find nearest dirty cell using A* heuristic
#         min_cost = float('inf')
#         nearest_dirty = None
        
#         for dirty in problem.initial_state:
#             if dirty.status == 'dirty':
#                 cost = robot.g + problem.heuristic(robot, dirty) + dirty.g
#                 if cost < min_cost:
#                     min_cost = cost
#                     nearest_dirty = dirty
        
#         if nearest_dirty is None:
#             break
            
#         # Move to nearest dirty cell
#         while (robot.position.x != nearest_dirty.position.x or 
#                robot.position.y != nearest_dirty.position.y):
            
#             # Determine possible actions
#             possible_acts = problem.actions(robot)
#             if isinstance(possible_acts, list) and action in possible_acts:
#                 robot = problem.result(robot, action)
#                 path.append(action)
#                 total_cost += 1
#             else:
#                 # If diagonal movement not possible, try cardinal directions
#                 if 'Right' in possible_acts and dy > 0:
#                     robot = problem.result(robot, 'Right')
#                     path.append('Right')
#                 elif 'Left' in possible_acts and dy < 0:
#                     robot = problem.result(robot, 'Left')
#                     path.append('Left')
#                 elif 'Down' in possible_acts and dx > 0:
#                     robot = problem.result(robot, 'Down')
#                     path.append('Down')
#                 elif 'Up' in possible_acts and dx < 0:
#                     robot = problem.result(robot, 'Up')
#                     path.append('Up')
#                 total_cost += 1
        
#         # Clean the cell, not increase by 1 cost for each movement of dirty cells, not store the heuristic cost
#         if problem.actions(robot) == 'Suck':
#             robot = problem.result(robot, 'Suck')
#             path.append('Suck')
#             total_cost += 1
#             nearest_dirty.status = 'clean'
            
#     return path, total_cost

In [210]:
def astar(problem: VacuumProblem): 
    """
    A* algorithm for vacuum cleaner problem
    Returns the path taken by the robot and total cost
    """
    robot = problem.robot_inital
    dirtycells = problem.initial_state
    total_cost = 0
    cleanedcells = []
    path = []

    while len(dirtycells) != 0:

        #Choose the nearest dirty cell
        dirty_min = float('inf')
        nearest_dirty = None
        for dirty in problem.initial_state:
            cost = robot.g + problem.heuristic(robot, dirty) 
            if cost < dirty_min:
                dirty_min = cost
                nearest_dirty = dirty
        
        #Find possible actions
        if (robot.position.x == nearest_dirty.position.x) and (robot.position.y == nearest_dirty.position.y):
            possible_acts = 'Suck'
        else:
            possible_acts = problem.actions(robot)  

        #Find best move
        bestact = None
        if isinstance(possible_acts, list): ##Not at the dirty cell
            action_min = float('inf')
            for action in possible_acts:
                newstate = problem.result(robot, action)
                cost = newstate.g + problem.heuristic(newstate, nearest_dirty) #moving cost and heuristic cost
                if cost < action_min:
                    action_min = cost
                    bestact = action
        else:
            bestact = 'Suck'
        
        #Do the action
        if bestact == 'Suck':
            cleaned = problem.result(nearest_dirty, bestact)
            cleanedcells.append(cleaned)
            robot.g += cleaned.g  #cost of sucking
            total_cost += robot.g
            dirtycells.remove(nearest_dirty)
            path.append(bestact)
        else: 
            robot = problem.result(robot, bestact) #moving
            robot.g += 1  #moving cost
            robot.h += problem.heuristic(robot, nearest_dirty)
            robot.f += robot.g + robot.h
            total_cost += robot.f
            path.append(bestact)
        #Update the cost of to-be-cleaned dirty cells
        for remain in dirtycells:
            remain.g += 1
    
    return path, total_cost#, cleanedcells



# Giải bài toán hút bụi bằng A*

In [211]:
#Create the world (matrix)
m, n = 10, 8
world = np.zeros((m,n), dtype = int)
#Create dirty cells
dirt1 = Cell(position=Position(4, 0), status='dirty')
dirt2 = Cell(position=Position(m-1, 0), status='dirty')
dirt3 = Cell(position=Position(2, 2), status = 'dirty')
dirt4 = Cell(position=Position(2, 4), status='dirty')
dirt5 = Cell(position=Position(4, n-1), status = 'dirty')
dirty_initals = [dirt1, dirt2, dirt3, dirt4, dirt5]

#Create robot
agent = Cell(position=Position(5, 3), status='start')


In [212]:
# new_position = Position(robot.position.x, robot.position.y)
# newstate = Cell(position=new_position, status=robot.status, g=robot.g)
# newstate = problem.result(newstate, action)

In [213]:
vacuum_world = VacuumProblem(matrix=world, robot_inital=agent, initial_state=dirty_initals)

In [214]:
solution, cost  = astar(vacuum_world)